In [11]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier

In [35]:
# Gray Scale images are collected from the Bhoonidhi Website liscened by ISRO.
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (256, 256))  # Resize for consistency
            images.append(img)
    return images

In [43]:
# Reference Color Image was collected from the Bhoonidhi website liscened by ISRO.
def load_color_reference_images(folder):
    color_images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        img = cv2.imread(filepath)
        if img is not None:
            img = cv2.resize(img, (256, 256))  # Resize for consistency
            color_images.append(img)
    return color_images

In [34]:
def colorize_image_knn(grayscale_image, color_reference_images, k=5):
    height, width = grayscale_image.shape
    knn = KNeighborsClassifier(n_neighbors=k)

    # Prepare training data from color reference images
    training_data = []
    labels = []
    
    for color_image in color_reference_images:
        lab_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2LAB)
        L, A, B = cv2.split(lab_image)
        
        # Reshape to 1D arrays and store L, A, B values
        for i in range(height):
            for j in range(width):
                training_data.append([L[i, j]])  # Use L-channel as the input
                labels.append([A[i, j], B[i, j]])  # A and B channels as target labels

    training_data = np.array(training_data)
    labels = np.array(labels)

    # Train the KNN model
    knn.fit(training_data, labels)

    # Predict the A and B channels for the grayscale image
    L_channel = grayscale_image.flatten().reshape(-1, 1)
    predicted_ab = knn.predict(L_channel)

    # Reconstruct the colorized LAB image
    A_channel = predicted_ab[:, 0].reshape(height, width)
    B_channel = predicted_ab[:, 1].reshape(height, width)
    colorized_lab = cv2.merge([grayscale_image, A_channel, B_channel])

    # Convert LAB back to RGB
    colorized_rgb = cv2.cvtColor(colorized_lab, cv2.COLOR_LAB2BGR)
    return colorized_rgb

In [33]:
def process_and_colorize_images(input_folder, reference_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load grayscale images and color reference images
    grayscale_images = load_images_from_folder(input_folder)
    color_reference_images = load_color_reference_images(reference_folder)

    for i, grayscale_image in enumerate(grayscale_images):
        # Colorize using KNN
        colorized_image = colorize_image_knn(grayscale_image, color_reference_images, k=5)
        
        # Save the colorized image
        output_filename = os.path.join(output_folder, f'colorized_image_{i}.jpg')
        cv2.imwrite(output_filename, colorized_image)
        print(f"Colorized image saved: {output_filename}")


In [32]:
input_folder = r'D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\black_white'
reference_folder = r'D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\color'
output_folder = r'D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output'


In [29]:
process_and_colorize_images(input_folder, reference_folder, output_folder)

Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output\colorized_image_0.jpg
Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output\colorized_image_1.jpg
Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output\colorized_image_2.jpg
Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output\colorized_image_3.jpg
Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output\colorized_image_4.jpg
Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output\colorized_image_5.jpg
Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output\colorized_image_6.jpg
Colorized image saved: D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\coloriz

In [36]:
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

In [37]:
def load_images_from_folder(folder, color=True):
    images = []
    for filename in sorted(os.listdir(folder)):  # Ensure sorted order
        filepath = os.path.join(folder, filename)
        if color:
            img = cv2.imread(filepath)  # Color image for ground truth
        else:
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Grayscale image for input
        if img is not None:
            img = cv2.resize(img, (256, 256))  # Resize for consistency
            images.append(img)
    return images

In [38]:
def evaluate_colorization_batch(colorized_folder, ground_truth_folder):
    colorized_images = load_images_from_folder(colorized_folder, color=True)
    ground_truth_images = load_images_from_folder(ground_truth_folder, color=True)

    mse_a_total, mse_b_total, ssim_a_total, ssim_b_total = 0, 0, 0, 0
    count = len(colorized_images)

    for colorized_img, ground_truth_img in zip(colorized_images, ground_truth_images):
        # Convert both images to LAB color space
        colorized_lab = cv2.cvtColor(colorized_img, cv2.COLOR_BGR2LAB)
        ground_truth_lab = cv2.cvtColor(ground_truth_img, cv2.COLOR_BGR2LAB)

        # Split LAB channels
        _, A_c, B_c = cv2.split(colorized_lab)
        _, A_g, B_g = cv2.split(ground_truth_lab)

        # Calculate MSE for A and B channels
        mse_a = mean_squared_error(A_c, A_g)
        mse_b = mean_squared_error(B_c, B_g)
        
        # Calculate SSIM for A and B channels
        ssim_a = ssim(A_c, A_g)
        ssim_b = ssim(B_c, B_g)

        # Accumulate scores
        mse_a_total += mse_a
        mse_b_total += mse_b
        ssim_a_total += ssim_a
        ssim_b_total += ssim_b

    # Calculate average MSE and SSIM for all images
    avg_mse_a = mse_a_total / count
    avg_mse_b = mse_b_total / count
    avg_ssim_a = ssim_a_total / count
    avg_ssim_b = ssim_b_total / count

    return {
        "Average MSE A-channel": avg_mse_a,
        "Average MSE B-channel": avg_mse_b,
        "Average SSIM A-channel": avg_ssim_a,
        "Average SSIM B-channel": avg_ssim_b
    }


In [40]:
colorized_folder = r"D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\colorized_output"
ground_truth_folder = r"D:\PAVAN\SAR_Image_Colorization_Pairs\SAR_Image_Colorization_Pairs\black_white"

In [41]:
results = evaluate_colorization_batch(colorized_folder, ground_truth_folder)
print("Batch Evaluation Results:", results)

Batch Evaluation Results: {'Average MSE A-channel': 61.03204895019531, 'Average MSE B-channel': 110.64799133300781, 'Average SSIM A-channel': 0.6938501129004656, 'Average SSIM B-channel': 0.8317735929691795}
